<a href="https://colab.research.google.com/github/StarBlossom99/Pytorch_Study/blob/main/2022_0113/pytorch_study_neuralnetwork.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
import os
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

In [8]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f'Using {device} device')

Using cuda device


In [9]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

flatten()은 다차원 배열을 1차원 배열로 변환

In [10]:
model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


In [23]:
X = torch.rand(1, 28, 28, device=device)
logits = model(X)
pred_probab = nn.Softmax(dim=1)(logits)
y_pred = pred_probab.argmax(1)
print(f"Predicted class: {y_pred}")

Predicted class: tensor([4], device='cuda:0')


random 하게 1* 28 * 28 의 값을 생성하고, device에 올린다음 model 함수를 통해 logits값을 구한다. 그 값을 softmax에 넣어서 어떤 class에 가장 가까운지 찾는다.

In [12]:
input_image = torch.rand(3, 28, 28)
print(input_image.size())

torch.Size([3, 28, 28])


In [13]:
flatten = nn.Flatten()
flat_image = flatten(input_image)
print(flat_image.size())

torch.Size([3, 784])


In [14]:
layer1 = nn.Linear(in_features=28*28, out_features=20)
hidden1 = layer1(flat_image)
print(hidden1.size())

torch.Size([3, 20])


layer를 통과한 뒤에 784에서 20으로 줄어든 모습

In [15]:
print(f"Before ReLU: {hidden1}\n\n")
hidden1 = nn.ReLU()(hidden1)
print(f"After ReLU: {hidden1}")

Before ReLU: tensor([[-0.0760, -0.2309,  0.1111,  0.3498, -0.4519, -0.2371, -0.2027,  0.1807,
         -0.4076,  0.1489,  0.1223,  0.1759, -0.0500, -0.2466,  0.0857, -0.2605,
          0.2504, -0.3415, -0.8098,  0.0458],
        [ 0.2214, -0.3312,  0.1217,  0.0117, -0.3884,  0.3854, -0.6269, -0.2166,
         -0.3893,  0.2119,  0.5174,  0.5277, -0.1911, -0.1036,  0.3012, -0.0190,
          0.2722, -0.3918, -0.4915,  0.1136],
        [ 0.2902, -0.2185,  0.5792,  0.1920, -0.3852,  0.0621, -0.3546, -0.1854,
         -0.0582,  0.0390,  0.5793,  0.6793, -0.0739, -0.3054,  0.1945,  0.0579,
          0.2594,  0.0999, -0.6539,  0.1643]], grad_fn=<AddmmBackward0>)


After ReLU: tensor([[0.0000, 0.0000, 0.1111, 0.3498, 0.0000, 0.0000, 0.0000, 0.1807, 0.0000,
         0.1489, 0.1223, 0.1759, 0.0000, 0.0000, 0.0857, 0.0000, 0.2504, 0.0000,
         0.0000, 0.0458],
        [0.2214, 0.0000, 0.1217, 0.0117, 0.0000, 0.3854, 0.0000, 0.0000, 0.0000,
         0.2119, 0.5174, 0.5277, 0.0000, 0.0000, 0.30

ReLU 함수 결과

In [19]:
seq_modules = nn.Sequential(
    flatten,
    layer1,
    nn.ReLU(),
    nn.Linear(20, 10)
)
input_image = torch.rand(3,28,28)
logits = seq_modules(input_image)

In [20]:
softmax = nn.Softmax(dim=1)
pred_probab = softmax(logits)

In [21]:
print("Model structure: ", model, "\n\n")

for name, param in model.named_parameters():
    print(f"Layer: {name} | Size: {param.size()} | Values : {param[:2]} \n")

Model structure:  NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
) 


Layer: linear_relu_stack.0.weight | Size: torch.Size([512, 784]) | Values : tensor([[-0.0153,  0.0141,  0.0038,  ...,  0.0287,  0.0232, -0.0219],
        [ 0.0252,  0.0100,  0.0053,  ...,  0.0216,  0.0222,  0.0238]],
       device='cuda:0', grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.0.bias | Size: torch.Size([512]) | Values : tensor([ 0.0324, -0.0350], device='cuda:0', grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.2.weight | Size: torch.Size([512, 512]) | Values : tensor([[ 0.0406,  0.0137,  0.0343,  ...,  0.0248, -0.0156,  0.0244],
        [ 0.0053,  0.0215,  0.0182,  ...,  0.0351, -0.0074, -0.0413]],
       device='cuda:0', grad_fn=<